In [66]:
# Import Standard Libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.api as sm
from statsmodels.tsa.stattools import ccf

# Import Feature Engineering/ML Libraries
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import f1_score, confusion_matrix, mean_absolute_percentage_error, r2_score, mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from scipy.stats import zscore
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from itertools import product
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


# Suppress warnings
warnings.filterwarnings("ignore")

In [67]:
#Keeping Track of Input Variables
path = '/Users/angelobenedicto/Documents/Commodities Hedger/Commodities'
target = 'CHK'
today = date.today()
hedge_period = 12
tts = 0.2

In [68]:
#Get List of Commodities
def list_csv_files(folder_path):
    csv_files = []
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            csv_files.append(file)
    return csv_files

comms = list_csv_files(path)

In [69]:
#Create Total Comm DF
total_comm_df = pd.read_csv(f'/Users/angelobenedicto/Documents/Commodities Hedger/Commodities/{comms[0]}')

for item in range(len(comms)):
    if item == 0:
        pass
    else:
        comm_df = pd.read_csv(f'/Users/angelobenedicto/Documents/Commodities Hedger/Commodities/{comms[item]}')
        total_comm_df = total_comm_df.merge(comm_df, how='outer', on='DATE')

#Create Date Time For Time esries Analysis
total_comm_df['DATE'] = pd.to_datetime(total_comm_df['DATE'])

#Fetch Manually Maintained Code Lookup
comm_lookup = pd.read_excel('/Users/angelobenedicto/Documents/Commodities Hedger/Commodities/CommLookUp.xlsx')

#Get List of Columns
comms = list(total_comm_df.columns)
comms.remove('DATE')

#Get FRED Short Code
short_codes = pd.DataFrame(comms).merge(comm_lookup[['FRED Ticker', 'Code']], how='left', left_on=0, right_on='FRED Ticker')
short_codes = list(short_codes['Code'])

#Add 'DATE' to column names
short_codes.insert(0, 'DATE')

#Rename Columns to Short Codes
total_comm_df.columns=short_codes

In [70]:
# Determine the complete monthly date range from the earliest to the latest date in total_comm_df
date_range = pd.date_range(start=total_comm_df['DATE'].min(), end=total_comm_df['DATE'].max(), freq='MS')

# Set the 'DATE' column as the index for reindexing
total_comm_df.set_index('DATE', inplace=True)

# Reindex the dataframe with the complete monthly date range, filling missing values as NaN
continuous_total_comm_df = total_comm_df.reindex(date_range)

# If you want to have 'DATE' as a column instead of the index
continuous_total_comm_df.reset_index(inplace=True)
continuous_total_comm_df.rename(columns={'index': 'DATE'}, inplace=True)

#Turn DF into Time Series
continuous_total_comm_df.set_index('DATE', inplace=True)

#Filter Out Dates Without Target
start_date = continuous_total_comm_df[[target]].dropna().index.min()
end_date = continuous_total_comm_df[[target]].dropna().index.max()
continuous_total_comm_df = continuous_total_comm_df[start_date:end_date]

In [71]:
#Quick Check For Missing Dates

# Assuming continuous_total_comm_df is your DataFrame
start_date = continuous_total_comm_df.index.min()
end_date = continuous_total_comm_df.index.max()

# Generate a complete range of monthly dates from start to end
complete_date_range = pd.date_range(start=start_date, end=end_date, freq='MS')

# Find missing dates by checking which dates in the complete range are not in the DataFrame's index
missing_dates = complete_date_range.difference(continuous_total_comm_df.index)

print("Missing Dates:")
print(missing_dates)

#Drop all NaN Values
continuous_total_comm_df.dropna(inplace=True)

Missing Dates:
DatetimeIndex([], dtype='datetime64[ns]', freq='MS')


In [72]:
# Convert object columns that should be numeric
for col in continuous_total_comm_df.columns:  # List other columns if needed
    continuous_total_comm_df[col] = pd.to_numeric(continuous_total_comm_df[col], errors='coerce')

# Function to impute values based on the described logic
def impute_values(df, column):
    for i, value in enumerate(df[column]):
        if pd.isnull(value):  # Check if value is NaN (formerly ".")
            prev_val = df[column].iloc[:i].dropna().tail(1).values  # Value before the NaN
            next_val = df[column].iloc[i+1:].dropna().head(1).values  # Value after the NaN

            if prev_val.size > 0 and next_val.size > 0:
                # Average of the period before and the period after
                df.at[df.index[i], column] = np.mean([prev_val[0], next_val[0]])
            else:
                # Impute with the mean of that year
                year = df.index[i].year
                yearly_mean = df[column][df.index.year == year].mean()
                df.at[df.index[i], column] = yearly_mean

# Apply the imputation function to each column that requires it
for col in continuous_total_comm_df.columns:
    if continuous_total_comm_df[col].dtype == 'float64':  # Assuming you want to apply this to float columns
        impute_values(continuous_total_comm_df, col)

In [73]:
#Get Rolling Values
for comm in continuous_total_comm_df.columns:
    for period in [3, 6, 9, 12]:
        continuous_total_comm_df[f'{comm} Rolling {period}M'] = continuous_total_comm_df[comm].rolling(window=period).mean()
        
#Get Rolling Growth Values
for comm in continuous_total_comm_df.columns:
    for period in [3, 6, 9, 12]:
        continuous_total_comm_df[f'{comm} M-{period}'] = continuous_total_comm_df[comm].shift(+period)
        continuous_total_comm_df[f'{comm} M+{period} Growth'] = continuous_total_comm_df[comm] / continuous_total_comm_df[f'{comm} M-{period}'] - 1

#Drop NaN Values
continuous_total_comm_df.dropna(inplace=True)

#Create 3M Future Value of Target
continuous_total_comm_df[f'{target} Future Price'] = continuous_total_comm_df[target].shift(-hedge_period)

#Get Price Increases
continuous_total_comm_df['Increase'] = np.where(continuous_total_comm_df[f'{target} Future Price'] > continuous_total_comm_df[target], 1, 0)

#Create lin_reg_df
lin_reg_df = continuous_total_comm_df.drop('Increase', axis=1)

#Drop Target Future Price
continuous_total_comm_df.drop(f'{target} Future Price', axis=1, inplace=True)

In [74]:
#Notes

# 1. Conduct Retracement Analysis -> quick walk through with andrew
# 2. Take into account inflation, everything is inclined to grow
# 3. Conduct a volume sold analysis - try to see if this has an effect on price
# 4. Explore signals, conventional trend analytics -> Can I detect a dead cat bounce, bull flag analysis, 
# 5. Check Random Forest to see which branches were cut first
# 6. Conduct PCA Analysis to determine most important features

In [75]:
def evaluate_model(model_name, y_true, y_pred):
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    print(f"{model_name} F1 Score: {f1}")
    print(f"{model_name} Confusion Matrix:")
    print(f"True Negatives (TN): {tn}")
    print(f"False Positives (FP): {fp}")
    print(f"False Negatives (FN): {fn}")
    print(f"True Positives (TP): {tp}\n")
    return f1

def run_multi_model(val_date, run_gradient_boost):
    val_date_ts = datetime.strptime(val_date, '%Y-%m')
    val_date_plus_one_month = val_date_ts + relativedelta(months=+1)
    val_date_next = val_date_plus_one_month.strftime('%Y-%m')

    df = continuous_total_comm_df[:val_date]
    X = df.drop('Increase', axis=1)
    y = df['Increase']
    scaler = StandardScaler().fit(X)
    X_scaled = scaler.transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=tts, random_state=42)

    f1_scores = []
    
    print('Test Set Results')
    print('')
    
    # Logistic Regression
    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(X_train, y_train)
    y_pred_lr = lr_model.predict(X_test)
    f1_lr = evaluate_model("Logistic Regression", y_test, y_pred_lr)
    f1_scores.append(["Logistic Regression", f1_lr])

    # XGBoost
    xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    xgb_model.fit(X_train, y_train)
    y_pred_xgb = xgb_model.predict(X_test)
    f1_xgb = evaluate_model("XGBoost", y_test, y_pred_xgb)
    f1_scores.append(["XGBoost", f1_xgb])

    if run_gradient_boost:
        # Gradient Boosting with Grid Search
        parameters = {'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7], 'n_estimators': [100, 200]}
        gb_model = GridSearchCV(estimator=GradientBoostingClassifier(), param_grid=parameters, scoring='f1', cv=5)
        gb_model.fit(X_train, y_train)
        best_gb_model = gb_model.best_estimator_
        y_pred_gb = best_gb_model.predict(X_test)
        f1_gb = evaluate_model("Gradient Boosting (Best)", y_test, y_pred_gb)
        f1_scores.append(["Gradient Boosting (Best)", f1_gb])

    # Random Forest
    rf_model = RandomForestClassifier()
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    f1_rf = evaluate_model("Random Forest", y_test, y_pred_rf)
    f1_scores.append(["Random Forest", f1_rf])

    top_model, top_f1 = max(f1_scores, key=lambda x: x[1])

    forecast_model = {
        "Logistic Regression": lr_model,
        "XGBoost": xgb_model,
        "Random Forest": rf_model
    }[top_model]

    val_df = continuous_total_comm_df[val_date_next:'2023-02']
    X_val = val_df.drop('Increase', axis=1)
    y_val = val_df['Increase']
    X_val_scaled = scaler.transform(X_val)
    y_val_preds = forecast_model.predict(X_val_scaled)
    
    print('Hold Out Set Results:')
    print('')

    f1_val = evaluate_model(top_model, y_val, y_val_preds)

    return y_val_preds, y_val, val_df


In [76]:
#Run Logistic Regression
y_val_preds, y_val, val_df = run_multi_model(val_date='2022-02', run_gradient_boost=False) 

Test Set Results

Logistic Regression F1 Score: 0.912621359223301
Logistic Regression Confusion Matrix:
True Negatives (TN): 12
False Positives (FP): 6
False Negatives (FN): 3
True Positives (TP): 47

XGBoost F1 Score: 0.9019607843137256
XGBoost Confusion Matrix:
True Negatives (TN): 12
False Positives (FP): 6
False Negatives (FN): 4
True Positives (TP): 46

Random Forest F1 Score: 0.8686868686868686
Random Forest Confusion Matrix:
True Negatives (TN): 12
False Positives (FP): 6
False Negatives (FN): 7
True Positives (TP): 43

Hold Out Set Results:

Logistic Regression F1 Score: 0.8571428571428571
Logistic Regression Confusion Matrix:
True Negatives (TN): 0
False Positives (FP): 0
False Negatives (FN): 3
True Positives (TP): 9



In [77]:
# Load the data
data = lin_reg_df  # Assuming the first column is a datetime index

# List of features to analyze
features = ['SOY', 'CHK', 'POT', 'WTI', 'GAS', 'PAY', 'ELE', 'WHT', 'BEF', 'CRN']
target = 'CHK Future Price'  # Assuming this is your target variable

# Maximum number of lags to check
lag_max = 12

# Prepare a DataFrame to store CCF results
ccf_results = pd.DataFrame(index=features, columns=[f'Lag {i}' for i in range(lag_max)] + ['Average of Lags'])

# Ensure the data is not missing and fill or interpolate if necessary
data = data[features + [target]].dropna()

for feature in features:
    # Compute Cross-Correlation Function (CCF)
    ccf_values = ccf(data[feature], data[target], adjusted=False)[:lag_max]
    
    # Assign the results to the DataFrame using .iloc for correct slicing
    ccf_results.loc[feature, :f'Lag {lag_max-1}'] = ccf_values
    
    # Compute the average of the lags and assign to the 'Average of Lags' column
    ccf_results.loc[feature, 'Average of Lags'] = np.mean(ccf_values)

# Display the results table
ccf_results.sort_values(by='Average of Lags', ascending=False)

,Lag 0,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7,Lag 8,Lag 9,Lag 10,Lag 11,Average of Lags
CHK,0.947848,0.940008,0.930471,0.920403,0.910646,0.902902,0.895114,0.885583,0.878596,0.870978,0.864201,0.859102,0.900488
BEF,0.931393,0.924204,0.916113,0.908179,0.899368,0.891441,0.883095,0.873192,0.863733,0.853217,0.843804,0.835375,0.88526
PAY,0.953788,0.941732,0.92833,0.915663,0.902253,0.88995,0.878192,0.864776,0.852855,0.839,0.825724,0.813811,0.88384
ELE,0.919554,0.909917,0.899249,0.887888,0.875449,0.864044,0.853431,0.842534,0.834131,0.824215,0.814525,0.805196,0.860844
SOY,0.822226,0.813923,0.80303,0.792003,0.781991,0.774009,0.76833,0.760572,0.754708,0.744423,0.732428,0.721671,0.772443
CRN,0.737674,0.727714,0.714709,0.69977,0.683702,0.666912,0.651528,0.635082,0.620582,0.603216,0.585225,0.570377,0.658041
WHT,0.721158,0.712795,0.701497,0.68862,0.674513,0.659996,0.648077,0.633874,0.62,0.603339,0.584933,0.570184,0.651582
WTI,0.649,0.638402,0.626165,0.614702,0.602254,0.588811,0.576389,0.561836,0.545416,0.527785,0.512456,0.498474,0.578474
POT,0.499968,0.504534,0.505968,0.506373,0.500917,0.499073,0.498426,0.495864,0.492607,0.48529,0.476146,0.462589,0.49398
GAS,-0.000001,-0.002675,-0.006403,-0.010359,-0.015595,-0.022942,-0.030128,-0.040478,-0.050085,-0.057283,-0.065277,-0.074768,-0.031333


In [78]:
ccf_results.to_excel('ccf_results.xlsx')